In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import ase
from ase import Atoms, Atom
from prepare_data import create_grid, deserialize_df, create_distance_matrices, create_expanded_distance_matrix

In [2]:
df0 = pd.read_pickle("./../Datasets/Cu_df2_1k.pkl.gzip", compression="gzip")
atoms = df0["ase_atoms"]
a = Atoms(atoms[800])
a.positions

array([[ 0.      ,  0.      ,  0.      ],
       [ 1.613117,  1.613117,  1.613117],
       [ 0.      ,  0.      ,  3.226233],
       [ 1.613117,  1.613117,  4.83935 ],
       [ 0.      ,  0.      ,  6.452467],
       [ 1.613117,  1.613117,  8.065583],
       [ 0.      ,  0.      ,  9.6787  ],
       [ 1.613117,  1.613117, 11.291817]])

In [3]:
df0 = deserialize_df(df0)
grid = create_grid(df0["ase_atoms"][800], 1)
grid

Adding neighbours: 100%|██████████| 8/8 [00:00<?, ?it/s]


[GridPoint(index: (0, 0, 0), atoms: Atom('Cu', [0.0, 0.0, 0.0], index=0)),
 GridPoint(index: (1, 1, 1), atoms: Atom('Cu', [1.613117, 1.613117, 1.613117], index=1)),
 GridPoint(index: (0, 0, 3), atoms: Atom('Cu', [0.0, 0.0, 3.2262330000000006], index=2)),
 GridPoint(index: (1, 1, 4), atoms: Atom('Cu', [1.613117, 1.613117, 4.8393500000000005], index=3)),
 GridPoint(index: (0, 0, 6), atoms: Atom('Cu', [0.0, 0.0, 6.452467], index=4)),
 GridPoint(index: (1, 1, 8), atoms: Atom('Cu', [1.613117, 1.613117, 8.065583], index=5)),
 GridPoint(index: (0, 0, 9), atoms: Atom('Cu', [0.0, 0.0, 9.678700000000001], index=6)),
 GridPoint(index: (1, 1, 11), atoms: Atom('Cu', [1.613117, 1.613117, 11.291817], index=7))]

In [4]:
distance_matrix, atom_numbers = create_distance_matrices(grid)
create_expanded_distance_matrix(distance_matrix)

Creating expanded distance matrix: 100%|██████████| 8/8 [01:22<00:00, 10.27s/it]


[array([[ 2.7940006 , -0.57735027, -0.57735027, -0.57735027],
        [ 3.226233  ,  0.        ,  0.        , -1.        ],
        [ 5.35010293, -0.3015114 , -0.3015114 , -0.904534  ],
        [ 6.452467  ,  0.        ,  0.        , -1.        ],
        [ 8.38199988, -0.19245013, -0.19245013, -0.96225043],
        [ 9.6787    ,  0.        ,  0.        , -1.        ],
        [11.51995764, -0.14002803, -0.14002803, -0.98019605]]),
 array([[ 2.7940006 ,  0.57735027,  0.57735027,  0.57735027],
        [ 2.79400003,  0.57735039,  0.57735039, -0.57735003],
        [ 3.226233  ,  0.        ,  0.        , -1.        ],
        [ 5.35010293,  0.3015114 ,  0.3015114 , -0.904534  ],
        [ 6.452466  ,  0.        ,  0.        , -1.        ],
        [ 8.38199988,  0.19245013,  0.19245013, -0.96225043],
        [ 9.6787    ,  0.        ,  0.        , -1.        ]]),
 array([[ 3.226233  ,  0.        ,  0.        ,  1.        ],
        [ 3.226234  ,  0.        ,  0.        , -1.        ],
    